# LSTM

In [ ]:
import tensorflow as tf
import pandas as pd

## Get IMdb data

In [ ]:
# 50,000 movie reviews labeled as "positive" or "negative"
imdb_data = pd.read_csv("IMDB Dataset.csv")

imdb_data["sentiment"] = imdb_data["sentiment"].map({"positive": 1, "negative":0})

imdb_train = imdb_data["review"].values[:25000]
imdb_test = imdb_data["review"].values[25000:]

imdb_train_labels = imdb_data["sentiment"].values[:25000]
imdb_test_labels = imdb_data["sentiment"].values[25000:]

In [ ]:
imdb_train[0]

## Tokenize

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer(num_words=10000, oov_token = "<oov>")
tokenizer.fit_on_texts(imdb_train)

# convert words to numbers and pad for the neural network to use as input
train_sequences = tokenizer.texts_to_sequences(imdb_train)
train_padded = pad_sequences(train_sequences, maxlen=120, truncating="post")

# tokenized using the word_index learned from the training data
testing_sequences = tokenizer.texts_to_sequences(imdb_test)
test_padded = pad_sequences(testing_sequences, maxlen=120, truncating="post")

## LSTM

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

### Train the model

In [ ]:
history = model.fit(train_padded, imdb_train_labels, batch_size=32,
                    epochs=5,
                    validation_split=0.1)

### Make predictions on text samples

In [ ]:
# predict on a positive sample

pos_text_to_predict = ["The movie was cool. The animation and the graphics were out of this world. I would recommend this movie."]

train_sequences = tokenizer.texts_to_sequences(pos_text_to_predict)
pos_padded = pad_sequences(train_sequences, maxlen=120, truncating="post")

#  make prediction
prediction = model.predict(pos_padded)

print(prediction)

In [ ]:
# predict on a negative sample

neg_text_to_predict = ["The movie was not good. The animation and the graphics were terrible. I would not recommend this movie."
]

train_sequences = tokenizer.texts_to_sequences(neg_text_to_predict)
neg_padded = pad_sequences(train_sequences, maxlen=120, truncating="post")

#  make prediction
prediction = model.predict(neg_padded)

print(prediction)

### Plot learning curves

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+ string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')